In [ ]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 8000
MAXLEN = 500

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          256000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 264,353
Trainable params: 264,353
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 64s 102ms/step - loss: 0.4330 - acc: 0.8015 - val_loss: 0.2918 - val_acc: 0.8826
Epoch 2/10
625/625 [==============================] - 63s 100ms/step - loss: 0.2753 - acc: 0.8936 - val_loss: 0.3043 - val_acc: 0.8806
Epoch 3/10
625/625 [==============================] - 62s 100ms/step - loss: 0.2347 - acc: 0.9133 - val_loss: 0.2935 - val_acc: 0.8760
Epoch 4/10
625/625 [==============================] - 63s 100ms/step - loss: 0.2108 - acc: 0.9205 - val_loss: 0.2930 - val_acc: 0.8784
Epoch 5/10
625/625 [==============================] - 62s 100ms/step - loss: 0.1920 - acc: 0.9294 - val_loss: 0.5816 - val_acc: 0.8116
Epoch 6/10
625/625 [==============================] - 62s 99ms/step - loss: 0.1794 - acc: 0.9335 - val_loss: 0.2932 - val_acc: 0.8922
Epoch 7/10
625/625 [==============================] - 62s 100ms/step - loss: 0.1664 - acc: 0.9389 - val_loss: 0.2868 - val_acc: 0.8878
Epoch 8/10
625/625 [==============================] - 62

In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 23s 30ms/step - loss: 0.3533 - acc: 0.8758
[0.3532625734806061, 0.875760018825531]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
model.save('/content/drive/My Drive/sentiment_analysis_model_new.h5')

In [ ]:
model2= tf.keras.models.load_model('/content/drive/My Drive/sentiment_analysis_model_new.h5')

In [ ]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,MAXLEN))
  pred[0] = encoded_text
  result = model2.predict(pred)
  if result[0]>0.5:
    print("Positive Review") 
  else:
    print("Negative Review") 
  #print(result[0])

predict("That movie was great! really loved it and would watch it again because it was amazingly great")
predict("That movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched")


Positive Review
Negative Review
